In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings("ignore")



In [2]:
# 讀取SizeData
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\2023Q1_size_data.xlsx", None)["工作表1"]

In [3]:
df["花紋"] = df["花紋"].map(lambda x: x[:4])

In [4]:
df["position_up"] = df["花紋"].map(lambda x: "New" if x == "R154" or x == "UAP1" else "No")

In [5]:
ch_key = ['FLT-REP', 'AG', 'A', 'FMC', '一般店', 'BSSC-FLT', 'CD', '中盤', 'DS',
       'TK', 'MM', 'FMC BFP', 'FLT', 'TTC', 'BFP', '量販店', 'OT',
       'FLT-BSSC', '加油站', '輪胎行', 'FT']
ch_value = ['FLT-REP', 'AG', 'BSSC', 'BSSC', 'BSSC', 'BSSC-FLT', 'BSSC', 'BSSC', 'BSSC', 'BSSC', 'BSSC', 'BSSC', 'BSSC-FLT', 'BSSC', 'BSSC', 'BSSC', 
            'FLT-REP', 'BSSC-FLT', 'BSSC', 'BSSC', 'FLT-REP']
ch_dict = dict(zip(ch_key, ch_value))

In [6]:
df["大通路"] = df["通路明細"].map(ch_dict)

In [7]:
df = df[df["預算/前實績"] == "23ACT"]

In [8]:
df["舊分類"] = df["舊分類"].map(lambda x: "LSR" if x == "LSR進口品" else x)

In [9]:
df = df[["年月", "舊分類", "商品代號", "大通路", "position_up", "NET金額", "條數"]]

In [10]:
df = df.fillna(0)

In [11]:
# 讀取RBA&C 2
rt2 = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\RT2\20230320_RT2_assumption v2.xlsx", None)["RBA simulator with OP"]


In [12]:
rt2["position_up"] = rt2["花紋"].map(lambda x: "New" if x == "R154" or x == "UAP1" else "No")

In [13]:
rt2 = rt2[["新年月", "Type", "PCODE", "大通路", "position_up", "NET Total", "數量"]]

In [14]:
rt2.columns = df.columns

In [75]:
# 篩選基準和對比資料
budget_df = df[(df["年月"] == 202302)]
actual_df = df[(df["年月"] == 202303)]

# budget_df = rt2[(rt2["年月"] ["年月"] == 202305)]
# actual_df = rt2[(rt2== 202306)]

In [77]:
# 複製貼上有要做販促的PCODE
promotion = pd.read_clipboard()["PCODE"].values.tolist()
budget_df["promotion"] = budget_df["商品代號"].map(lambda x: "Yes" if x in promotion else "No")

In [78]:
promotion = pd.read_clipboard()["PCODE"].values.tolist()
actual_df["promotion"] = actual_df["商品代號"].map(lambda x: "Yes" if x in promotion else "No")

In [79]:
final = pd.concat([budget_df, actual_df])

In [80]:
budget_asp = round(budget_df["NET金額"].sum() / budget_df["條數"].sum(), 1)
actual_asp = round(actual_df["NET金額"].sum() / actual_df["條數"].sum(), 1)

In [81]:
print(budget_asp, actual_asp)
print(round((actual_asp / budget_asp), 1))

9596.6 9022.9
0.9


In [82]:
# 預算和實績ASP差異
diff = actual_asp - budget_asp


In [83]:
diff

-573.7000000000007

In [84]:
# 創建一個空字典來存儲貢獻值
contribution = {}

In [85]:
# 計算客戶舊分類、大通路、position up和promotion對差異的貢獻值
# 將需要計算的欄位寫在list裡面然後建立一個loop
# 將list裡面每個欄位裡面的unique value計算基準和對比的asp
# 將對比和基準的asp做相減得到差異
# 如果差異大於0，儲存在positive的keys，小於0的存在negative的keys
for column in ['舊分類', '大通路', 'position_up', 'promotion']:
    for category in final[column].unique():
        category_budget = budget_df[budget_df[column] == category]
        category_budget_asp = category_budget["NET金額"].sum() / category_budget["條數"].sum()
        category_actual = actual_df[actual_df[column] == category]
        category_actual_asp = category_actual["NET金額"].sum() / category_actual["條數"].sum()
        category_difference = category_actual_asp - category_budget_asp
        # print(category_difference)
        contribution[f"{column}+{category}"] = category_difference if category_difference > 0 else 0
        contribution[f"{column}-{category}"] = category_difference if category_difference < 0 else 0

In [86]:
sum(contribution.values())

-4413.006775954634

In [87]:
# 總和貢獻值，計算所有keys的value，相加起來應該會等於基準和對比的asp的差異總和
contribution_sum = sum(contribution.values())

# 將貢獻值標準化為百分比，遍歷字典的每個key，然後計算每個value的百分比再乘上基準和對比的差異
contribution_percentages = {key: (value / contribution_sum) * diff for key, value in contribution.items()}

# 輸出結果
print(f"預算平均銷售NET價格：{budget_asp:.2f}")
print(f"實績平均銷售NET價格：{actual_asp:.2f}")
print(f"差異金額：{diff:.2f}")
for key, value in contribution_percentages.items():
    print(f"{key}的貢獻值：{value:.2f}")
# print(f"貢獻值總和:{sum(contribution_percentages.values()):.2f}")

預算平均銷售NET價格：9596.60
實績平均銷售NET價格：9022.90
差異金額：-573.70
舊分類+LSR的貢獻值：0.00
舊分類-LSR的貢獻值：-20.45
舊分類+TBR的貢獻值：0.00
舊分類-TBR的貢獻值：-35.65
大通路+FLT-REP的貢獻值：0.00
大通路-FLT-REP的貢獻值：-0.67
大通路+BSSC的貢獻值：0.00
大通路-BSSC的貢獻值：-98.41
大通路+BSSC-FLT的貢獻值：0.00
大通路-BSSC-FLT的貢獻值：-3.02
position_up+No的貢獻值：0.00
position_up-No的貢獻值：-90.23
position_up+New的貢獻值：0.00
position_up-New的貢獻值：-194.52
promotion+Yes的貢獻值：0.00
promotion-Yes的貢獻值：-68.55
promotion+No的貢獻值：0.00
promotion-No的貢獻值：-62.20


In [88]:
pro_mix = 0
ch_mix = 0
pos_up = 0
discount = 0
for key, value in contribution_percentages.items():
    if '舊分類' in key:
        pro_mix += value
    elif '大通路' in key:
        ch_mix += value
    elif 'position_up' in key:
        pos_up += value
    else:
        discount += value
        
    

In [89]:
[pro_mix, ch_mix, pos_up, discount]

[-56.09472231970848,
 -102.10468261466578,
 -284.7512071413696,
 -130.74938792425684]

In [90]:
print('product mix is {}'.format(round(pro_mix, 0)))
print('channel mix is {}'.format(round(ch_mix, 0)))
print('position up is {}'.format(round(pos_up, 0)))
print('discount is {}'.format(round(discount, 0)))

product mix is -56.0
channel mix is -102.0
position up is -285.0
discount is -131.0


In [53]:
pd.DataFrame([pos_up, 0, ch_mix, pro_mix, discount]).to_clipboard(index=False)